In [2]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [3]:
load_dotenv(override=True)
open_api_key = os.getenv("OPENAI_API_KEY")
if open_api_key:
    print(f'OPENAI_API_KEY: {open_api_key[:8]} exists')

OPENAI_API_KEY: sk-proj- exists


In [4]:
openai = OpenAI()
MODEL = "gpt-4o-mini"

In [5]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages)
    return response.choices[0].message.content
gr.ChatInterface(fn=chat, title="FlightAI Assistant",).launch()

/home/python/llm_projects/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [ ]:
ticket_price = {
    "usa": 200,
    "canada": 250,
    "uk": 300,
    "australia": 350
}

def get_ticket_prices(destination):
    print(f"Ticket price for {destination} called")
    city = destination.lower()
    return ticket_price.get(city, "Sorry, we do not fly to that destination.")
get_ticket_prices("Canada")

Ticket price for Canada called


250